In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Experiment 1
- 3 HH neurons
- reciprocal delayed synaptic connections
- intracellular constant current stimulation of $10 \frac{\mu A}{cm^2}$
- random inital phase

![alt text](img/exp1.png "Experiment 1")

### Configuration following Materials and Methods SI

$C = 1 \frac{\mu F}{cm^2}$

$g_{Na} = 120 \frac{mS}{cm^2}, g_K = 36 \frac{mS}{cm^2}, g_L = 0.3 \frac{mS}{cm^2}$

$E_{Na} = 50 mV, E_K = -77 mV, E_L = -54.5 mV$

$\tau_r = \tau_{ex} = 0.1 ms, \tau_d = \tau_{in} = 3 ms$

## Classes and functions

In [ ]:
from scipy.signal import find_peaks

def get_spike_times(ts, V):
    """
    Calculate times in ts of spikes in V.
    """
    spike_idx = find_peaks(V, height=(20., 150.), distance=7.)[0]
    return ts[spike_idx]

def get_idx(t):
    """
    Calculate array index from time value.
    """
    return np.around(t / step_size).astype(np.int32)

In [ ]:
class hh_neuron:
    """
    Our own implementation of the Hodgkin-Huxley neuron model
    """
    
    def __init__(self, my_id, config={}):
        self.C = 1.
        self.g_Na = 120.
        self.g_K = 36.
        self.g_L = .3
        self.E_Na = 50.
        self.E_K = -77.
        self.E_L = -54.5
        self.I_ext = 10.
        self.tau_d = 3.
        self.tau_r = .1
        self.sim_n = 1
        self.id = my_id
        
        if 'C' in config.keys():
            self.C = config['C']
        if 'g_Na' in config.keys():
            self.g_Na = config['g_Na']
        if 'g_K' in config.keys():
            self.g_K = config['g_K']
        if 'g_L' in config.keys():
            self.g_L = config['g_L']
        if 'E_Na' in config.keys():
            self.E_Na = config['E_Na']
        if 'E_K' in config.keys():
            self.E_K = config['E_K']
        if 'E_L' in config.keys():
            self.E_L = config['E_L']
        if 'I_ext' in config.keys():
            self.I_ext = config['I_ext']
        if 'tau_d' in config.keys():
            self.tau_d = config['tau_d']
        if 'tau_r' in config.keys():
            self.tau_r = config['tau_r']
        
        self.inc = []
        self.outc = []
        self.V = None
        self.m = None
        self.h = None
        self.n = None

    def alpha_m(self, V):
        """S5"""
        return .1 * (V + 40.) / (1. - np.exp(-(V + 40.) / 10.))

    def beta_m(self, V):
        """S6"""
        return 4. * np.exp(-(V + 65.) / 18.)

    def alpha_h(self, V):
        """S7"""
        return .07 * np.exp(-(V + 65.) / 20.)

    def beta_h(self, V):
        """S8"""
        return 1. / (1. + np.exp(-(V + 35.) / 10.))

    def alpha_n(self, V):
        """S9
        "/ 10" appears to be a typo in the paper.
        Other resources say "/ 100" """
        return ((V + 55.) / 100.) / (1. - np.exp((-.1) * (V + 55.)))

    def beta_n(self, V):
        """S10"""
        return .125 * np.exp(-(V + 65.) / 80.)

    def dm(self, V, m):
        """S2"""
        a = self.alpha_m(V)
        b = self.beta_m(V)

        return a * (1. - m) - b * m

    def dh(self, V, h):
        """S3"""
        a = self.alpha_h(V)
        b = self.beta_h(V)

        return a * (1. - h) - b * h

    def dn(self, V, n):
        """S4"""
        a = self.alpha_n(V)
        b = self.beta_n(V)

        return a * (1. - n) - b * n

    def dV(self, t, V, m, h, n):
        """S1"""
        I_syn = self.I_syn(t)
        
        return (- self.g_Na * m ** 3 * h * (V - self.E_Na) 
                - self.g_K * n ** 4 * (V - self.E_K) 
                - self.g_L * (V - self.E_L) 
                + self.I_ext + I_syn ) / self.C
    
    def setup(self, size, m0, h0, n0, V0):
        """
        Setup the neuron before simulation.
        """
        self.m = np.zeros((size))
        self.h = np.zeros((size))
        self.n = np.zeros((size))
        self.V = np.zeros((size))
        
        self.m[0] = m0
        self.h[0] = h0
        self.n[0] = n0
        self.V[0] = V0
    
    def register_in(self, connection):
        """
        Register incomming connection.
        """
        self.inc.append(connection)
        
    def register_out(self, connection):
        """
        Register outgoing connection.
        """
        self.outc.append(connection)
    
    def alpha(self, t):
        """S11"""
        # avoid overflow
        t[t < -.3] = -.3 
        return (np.exp(-t / tau_d) - np.exp(-t / tau_r)) / (tau_d - tau_r)
    
    def I_syn(self, t): 
        """S12"""
        idx = get_idx(t)
        I_syn = 0.
        
        for c in self.inc:
            
            # calculate alpha for each combination of t_spike and tau_l values
            st = get_spike_times(ts[:idx], c.neuron_a.V[:idx])
            if st.size != 0:
                t_spike_grid, tau_l_grid = np.meshgrid(st, c.tau_ls)
                t_spike_grid = np.reshape(t_spike_grid, (-1))
                tau_l_grid = np.reshape(tau_l_grid, (-1))
                alphas = self.alpha(t - t_spike_grid - tau_l_grid)

                # S12 summed up over all incomming connections
                I_syn -= g_max / N * np.sum(alphas) * (c.neuron_a.V[idx] - c.E_syn)
    
        return I_syn
                                    
    def heun_step(self, t):
        """
        Perform one step of Heun's method for time step t.
        """
        i = get_idx(t)
                                   
        # intermediate
        m_i = self.m[i-1] + step_size * self.dm(self.V[i-1], self.m[i-1])
        h_i = self.h[i-1] + step_size * self.dh(self.V[i-1], self.h[i-1])
        n_i = self.n[i-1] + step_size * self.dn(self.V[i-1], self.n[i-1])
        V_i = self.V[i-1] + step_size * self.dV(t-step_size, self.V[i-1], self.m[i-1], self.h[i-1], self.n[i-1])
                                   
        # final
        self.m[i] = self.m[i-1] + (step_size / 2.) * (self.dm(self.V[i-1], self.m[i-1]) + self.dm(V_i, m_i))
        self.h[i] = self.h[i-1] + (step_size / 2.) * (self.dh(self.V[i-1], self.h[i-1]) + self.dh(V_i, h_i))
        self.n[i] = self.n[i-1] + (step_size / 2.) * (self.dn(self.V[i-1], self.n[i-1]) + self.dn(V_i, n_i))
        self.V[i] = self.V[i-1] + (step_size / 2.) * (self.dV(t-step_size, self.V[i-1], self.m[i-1], self.h[i-1], self.n[i-1]) + self.dV(t, V_i, m_i, h_i, n_i))
     
    def get_phase(self, t):
        """
        Returns the angle of the polar coordinates
        of n(t) (~y) and V(t) (~x).
        """
        
        i = get_idx(t)
        
        # translate to origin according to the 
        # precalculated phase cycle
        V_transl = (self.V[i] - V_phase_mean) / (V_phase_max - V_phase_min)
        n_transl = (self.n[i] - n_phase_mean) / (n_phase_max - n_phase_min)
        
        # alpha = arctan(y / x), alpha in [-pi, +pi]
        # shift by pi to get alpha within [0, 2pi]
        return np.arctan(n_transl / V_transl) + np.pi

In [ ]:
class Connection():
    """
    Our own implementation of a neuronal synaptic connection.
    """
    
    def __init__(self, neuron_a, neuron_b, tau_l, E_syn):
        self.neuron_a = neuron_a
        self.neuron_b = neuron_b
        self.neuron_a.register_out(self)
        self.neuron_b.register_in(self)
        self.E_syn = E_syn
        
        if distribute_tau_l:
            theta = tau_l / k
            # draw from gamma distribution
            self.tau_ls = np.random.gamma(k, theta, (int(N)))
            # max out values to avoid overflow
            #self.tau_ls[self.tau_ls > 1.] = 1.
        else:
            self.tau_ls = np.full((int(N)), tau_l)

## Configuration

In [ ]:
np.random.seed(42)

# a name string for the simulation scenario
scenario = "8ms_inh"

# configuration of the Hodgkin-Huxley neurons
hh_config = {
    'C': 1.,
    'g_Na': 120.,
    'g_K': 36.,
    'g_L': .3,
    'E_Na': 50.,
    'E_K': -77.,
    'E_L': -54.5,
    'I_ext': 10.
}
# If False, neuron gamma will not develop a tonic firing mode.
gamma_firing = True

# alpha funciton parameters
tau_d = 3.
tau_r = .1

# axonal delays
# If True, tau_l will be randomly drawn from a gamma distribution.
distribute_tau_l = False
# number of samples
N = 1.
# fixed value or mean value for distribution
tau_l = 8.
# shape factor of distribution
k = 5.

# further synaptic configuration
E_syn = -80.
g_max = .05

# simulation config
t0 = 0.
t_end = 120.
step_size = .02

# initial values - will be set to a consistent state after a warm up interval
m0 = 0.05
h0 = 0.6
n0 = 0.32
V0 = -30.

# phase length estimation 
# phase length is always somewhere around 14.66 ms
phase_len_t = 14.66

# If True, data will be written to a file.
write_to_file = False

## Initialization

In [ ]:
ts = np.arange(t0, t_end, step_size)

gamma_config = hh_config
if not gamma_firing:
    gamma_config["I_ext"] = 0.

hh_alpha = hh_neuron(0, hh_config)
hh_beta = hh_neuron(1, hh_config)
hh_gamma = hh_neuron(2, gamma_config)
neurons = [hh_alpha, hh_beta, hh_gamma]

## Simulation

In [ ]:
def simulate(ts):
    """
    Simulate over time array ts.
    """
    for t in ts[1:]:
        for neuron in neurons:
            neuron.heun_step(t)

### Warm up

In [ ]:
# warm up interval
ts_wu = np.arange(0, phase_len_t * 3., step_size)
for neuron in neurons:
    neuron.setup(ts_wu.size, m0, h0, n0, V0)
simulate(ts_wu)

### Capture one phase as reference

Save one phase of n and V in order to extract phase dynamics for later use.

In [ ]:
# warmup is 3 phases long, so use the last third 
# to securely get data, which is already following the phase dynamics
start_idx = int(phase_len_t * 2 / step_size)
V_phase = neurons[0].V[start_idx:]
n_phase = neurons[0].n[start_idx:]

# calculate the centroid of the phase cycle
# use mid point because of diverging data density
V_phase_min = np.min(V_phase)
n_phase_min = np.min(n_phase)
V_phase_max = np.max(V_phase)
n_phase_max = np.max(n_phase)
V_phase_mean = (V_phase_max - V_phase_min) / 2. + V_phase_min
n_phase_mean = (n_phase_max - n_phase_min) / 2. + n_phase_min

# plot cycle without transformation
plt.plot(V_phase, n_phase, ".")
plt.plot(V_phase_mean, n_phase_mean, "x")
plt.legend(["Phase cycle", "Centroid"])
plt.ylabel("n(t)")
plt.xlabel("V(t)")
plt.title("Phase cycle")
plt.show()

# translate the phase cycle to the origin
V_phase -= V_phase_mean
n_phase -= n_phase_mean

# scale cycle for phase determination
V_phase /= V_phase_max - V_phase_min
n_phase /= n_phase_max - n_phase_min

# plot cycle with transformation
plt.plot(V_phase, n_phase, ".")
plt.plot(0., 0., "x")
plt.legend(["Phase cycle", "Centroid"])
plt.ylabel("n(t)")
plt.xlabel("V(t)")
plt.title("Phase cycle transformed")
plt.show()

### Re-initialize with phase-consistent values and start real simulation

In [ ]:
# cut first phase
phase_len = int(phase_len_t / step_size)
for neuron in neurons:    
    neuron.m = neuron.m[phase_len:]
    neuron.h = neuron.h[phase_len:]
    neuron.n = neuron.n[phase_len:]
    neuron.V = neuron.V[phase_len:]

# shift phases randomly and reset neurons with new random, but consistent states
for neuron in neurons:
    shift = int(np.random.rand() * phase_len_t / step_size)
    
    m0 = np.roll(neuron.m, shift)[0]
    h0 = np.roll(neuron.h, shift)[0]
    n0 = np.roll(neuron.n, shift)[0]
    V0 = np.roll(neuron.V, shift)[0]
    
    neuron.setup(ts.size, m0, h0, n0, V0)
    
# connect neurons
connections = []
connections.append(Connection(hh_alpha, hh_gamma, tau_l, E_syn))
connections.append(Connection(hh_gamma, hh_alpha, tau_l, E_syn))
connections.append(Connection(hh_beta, hh_gamma, tau_l, E_syn))
connections.append(Connection(hh_gamma, hh_beta, tau_l, E_syn))
    
# simulate
simulate(ts)

## Evaluation

In [ ]:
def p(t, m, n):
    """S16"""
    i = np.sqrt(-1 + 0j)
    return .5 * np.abs(np.exp(i * m.get_phase(t)) + np.exp(i * n.get_phase(t)))

### Graphs

In [ ]:
import matplotlib.patches as mpatches

plt.rcParams["figure.figsize"] = (20, 5)
plt.figure()

# extract spike times
sta = get_spike_times(ts, hh_alpha.V)
stb = get_spike_times(ts, hh_beta.V)

# calculate nearest neighbors of spikes
from scipy.spatial import KDTree
sta_rs = np.reshape(sta, (sta.shape[0], 1))
stb_rs = np.reshape(stb, (stb.shape[0], 1))
alpha_tree = KDTree(sta_rs)
nn_dist, nn_idx = alpha_tree.query(stb_rs, 1)

# plot alpha and beta to visualize synchrony
n_plot = neurons[:-1]
colors = plt.get_cmap('inferno').colors
c_step = int(len(colors) / len(n_plot))
handles = []
labels = ["alpha", "beta", "gamma"]
for i, neuron in enumerate(n_plot):
    c_idx = int(i * c_step + c_step / 2)
    plt.plot(ts, neuron.V, color=colors[c_idx])
    spike_times = get_spike_times(ts, neuron.V)
    for st in spike_times:
        plt.axvline(st, ls="--", color=colors[c_idx])
    handles.append(mpatches.Patch(color=colors[c_idx], label=labels[i]))
plt.title("Membrane potential")
plt.legend(handles=handles)
plt.ylabel("V_m")
plt.xlabel("t")
plt.show()

# plot some nearest neighbors to check nearest neighbor detection
plt.plot(ts, hh_alpha.V, color="blue")
plt.plot(ts, hh_beta.V, color="orange")
n_nn_samples = len(sta)-1
for i in range(n_nn_samples + 1):
    idx = np.random.randint(1, len(sta)-1)
    c_idx = int(i * (len(colors) - 1) / n_nn_samples)
    plt.axvline(stb[idx], color=colors[c_idx], ls="--")
    plt.axvline(sta[nn_idx[idx]], color=colors[c_idx], ls="--")
plt.title("Nearest neighbor detection")
plt.legend(handles=[
    mpatches.Patch(color="blue", label="alpha"), 
    mpatches.Patch(color="orange", label="beta")
])
plt.ylabel("V_m")
plt.xlabel("t")
plt.show()

# phase lengths
pla = []
for i in range(1, sta.shape[0]):
    pla.append(sta[i] - sta[i-1])

plb = []
for i in range(1, stb.shape[0]):
    plb.append(stb[i] - stb[i-1])

plt.plot(sta[1:], pla)
plt.plot(stb[1:], plb)
plt.legend(["aplha", "beta"])
plt.title("Time difference between spike i and spike i-1")
plt.xlabel("t")
plt.ylabel("dt")
plt.show()

# nearest spike neighbor
plt.plot(stb_rs[1:], nn_dist[1:])
plt.title("Distance from alphas spikes to their nearest neighbors in betas spikes")
plt.ylabel("dt")
plt.xlabel("t")
plt.show()

# synchrony measure
ps = []
ps = p(ts, neurons[0], neurons[1])
plt.plot(ts, ps)
plt.title("Synchrony measure p(t)")
plt.ylabel("p(t)")
plt.xlabel("t")
plt.show()

### Percistency

In [ ]:
# write data to persistent files
if write_to_file:
    from datetime import datetime
    import csv

    now = datetime.now().strftime("%H-%M-%S")
    with open("dumps/" + now + "_hh-neurons-config_" + scenario + ".csv", "w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        csvWriter.writerow(["N", "E_syn", "distribute_tau_l", "tau_l", "k", "I_ext_gamma", "t_end", "gamma_firing"])
        csvWriter.writerow([N, E_syn, distribute_tau_l, tau_l, k, hh_gamma.I_ext, t_end, gamma_firing])
    with open("dumps/" + now + "_hh-neurons_" + scenario + ".csv", "w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        for n in neurons:
            csvWriter.writerow(n.V)